In [10]:
!pip freeze > requirements.txt

In [1]:
import sklearn # Check Sklearn version
import xgboost
import flask
print(sklearn.__version__)
print(xgboost.__version__)
print(flask.__version__)

1.5.2
2.1.3
3.1.0


/tmp/ipykernel_156167/1998059267.py:6: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Flask 3.1. Use feature detection or 'importlib.metadata.version("flask")' instead.
  print(flask.__version__)


In [2]:
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import datetime
import time
import tarfile
import boto3
import pandas as pd

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'mainbucketrockhight5461' # Mention the created S3 bucket name here
print("Using bucket " + bucket)
# hi
print(f"sagemaker version: {sagemaker.__version__}")

/home/murivirg/work/github/sagemaker-tutorials/customer_case_using_container/env/lib/python3.13/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/26/25 14:09:02] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=247037;file:///home/murivirg/work/github/sagemaker-tutorials/customer_case_using_container/env/lib/python3.13/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=74855;file:///home/murivirg/work/github/sagemaker-tutorials/customer_case_using_container/env/lib/python3.13/site-packages/botocore/credentials.py#1352\1352]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/murivirg/.config/sagemaker/config.yaml


[03/26/25 14:09:03] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=983585;file:///home/murivirg/work/github/sagemaker-tutorials/customer_case_using_container/env/lib/python3.13/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=520875;file:///home/murivirg/work/github/sagemaker-tutorials/customer_case_using_container/env/lib/python3.13/site-packages/botocore/credentials.py#1352\1352]8;;\

Using bucket mainbucketrockhight5461
sagemaker version: 2.242.0


In [3]:
import pickle

with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [4]:
print(type(model))
print(model)

<class 'sklearn.pipeline.Pipeline'>
Pipeline(steps=[('processing',
                 <transformers.RawDataProcessor object at 0x71ca622dbe00>),
                ('slice_columns',
                 <transformers.DataSlicer object at 0x71ca61b26f90>),
                ('null_filling',
                 <transformers.NullFillTransformer object at 0x71ca61b27cb0>),
                ('model',
                 FitModel(folds=5,
                          hyper_parameters={'colsample_bytree': [0.6, 0.8],
                                            'gamma': [2], 'max_depth': [3],
                                            'min_child_weight': [3],
                                            'random_state': [1005],
                                            'subsample': [0.6, 0.8]}))])


In [5]:
print(model.get_params())

{'memory': None, 'steps': [('processing', <transformers.RawDataProcessor object at 0x71ca622dbe00>), ('slice_columns', <transformers.DataSlicer object at 0x71ca61b26f90>), ('null_filling', <transformers.NullFillTransformer object at 0x71ca61b27cb0>), ('model', FitModel(folds=5,
         hyper_parameters={'colsample_bytree': [0.6, 0.8], 'gamma': [2],
                           'max_depth': [3], 'min_child_weight': [3],
                           'random_state': [1005], 'subsample': [0.6, 0.8]}))], 'verbose': False, 'processing': <transformers.RawDataProcessor object at 0x71ca622dbe00>, 'slice_columns': <transformers.DataSlicer object at 0x71ca61b26f90>, 'null_filling': <transformers.NullFillTransformer object at 0x71ca61b27cb0>, 'model': FitModel(folds=5,
         hyper_parameters={'colsample_bytree': [0.6, 0.8], 'gamma': [2],
                           'max_depth': [3], 'min_child_weight': [3],
                           'random_state': [1005], 'subsample': [0.6, 0.8]}), 'model__folds'

In [6]:
import pandas as pd
import pickle

# Load the model
with open('model.pkl', 'rb') as f:
    pipeline = pickle.load(f)

# Assuming 'decline_v2a_debit' is one of the required features
input_data = pd.DataFrame({
    'timestamp': ['2023-05-01'],
    'in_data': ['{"yams_score":0.7,"north_star_metric":"5.5"}'],
    'decline_v2a_debit': [0.5],
    'days_since_sms_otp_success': [20],
    'days_since_receiver_first_seen': [100],
    'days_since_device_first_seen': [20],
    'dda_age_in_days': [100]# Add this and any other missing features
    # ... add all other required features ...
})

# Make a prediction
prediction = pipeline.predict(input_data)

In [7]:
print(prediction)

{'uncalibrated': array([[0.14639568, 0.8536043 ]], dtype=float32), 'calibrated': array([[0.60676062, 0.39323938]])}


In [9]:
import os
import sys
import pandas as pd
from flask import Flask

# Add the directory containing predictor.py and transformers.py to the Python path
sys.path.append(os.path.abspath('docker-folder/src'))
sys.path.append(os.path.abspath('.'))  # For transformers.py in the root directory

# Import the ScoringService and app from predictor.py without modifying it
from predictor_script import ScoringService, app

# Temporarily set the model path to the current directory where model.pkl is located
original_model_path = 'docker-folder/model'
test_model_path = 'docker-folder/model'  # model.pkl is in the root directory
os.environ['MODEL_PATH'] = test_model_path  # Optional, depending on predictor.py implementation

# Ensure the model loads successfully
model = ScoringService.get_model()
assert model is not None, "Model failed to load from model.pkl"

# Test the /ping endpoint
with app.test_client() as client:
    response = client.get('/ping')
    assert response.status_code == 200, f"Ping endpoint failed with status {response.status_code}"
    print("Ping endpoint test passed!")

# Prepare sample CSV input data with headers for /invocations
sample_data = pd.DataFrame({
    'timestamp': ['2023-05-01'],
    'in_data': ['{"yams_score":0.7,"north_star_metric":"5.5"}'],
    'decline_v2a_debit': [0.5],
    'days_since_sms_otp_success': [20],
    'days_since_receiver_first_seen': [100],
    'days_since_device_first_seen': [20],
    'dda_age_in_days': [100]
})
csv_data = sample_data.to_csv(index=False)

# Test the /invocations endpoint
with app.test_client() as client:
    response = client.post('/invocations', data=csv_data, content_type='text/csv')
    assert response.status_code == 200, f"Invocations endpoint failed with status {response.status_code}"
    predictions = response.get_data(as_text=True).strip()
    assert predictions, "No predictions returned from /invocations"
    print("Invocations endpoint test passed!")
    print(f"Predictions: {predictions}")

print("All tests completed successfully!")

Ping endpoint test passed!
Invoked with 1 records
Invocations endpoint test passed!
Predictions: [[0.14639568 0.8536043 ]]
[[0.60676062 0.39323938]]
All tests completed successfully!


In [10]:
s3 = boto3.client('s3')

# Upload the tar.gz file to S3
s3.upload_file("model.tar.gz", bucket, "models/model.tar.gz")
model_data = f"s3://{bucket}/models/model.tar.gz"

print(f"model data: {model_data}")

model data: s3://mainbucketrockhight5461/models/model.tar.gz


In [54]:
import requests
import pandas as pd
from io import StringIO

# Test the ping endpoint
print("Testing /ping endpoint...")
ping_url = "http://localhost:8081/ping"
try:
    response = requests.get(ping_url, timeout=5)
    response.raise_for_status()
    print(f"Ping successful! Status code: {response.status_code}")
except requests.RequestException as e:
    print(f"Ping failed: {e}")

# Test the invocations endpoint
print("\nTesting /invocations endpoint...")
invocations_url = "http://localhost:8081/invocations"

# Prepare sample CSV input data (similar to your successful test)
sample_data = pd.DataFrame({
    'timestamp': ['2023-05-01'],
    'in_data': ['{"yams_score":0.7,"north_star_metric":"5.5"}'],
    'decline_v2a_debit': [0.5],
    'days_since_sms_otp_success': [20],
    'days_since_receiver_first_seen': [100],
    'days_since_device_first_seen': [20],
    'dda_age_in_days': [100]
})
csv_data = sample_data.to_csv(index=False)

headers = {"Content-Type": "text/csv"}
try:
    response = requests.post(invocations_url, headers=headers, data=csv_data, timeout=5)
    response.raise_for_status()
    print("Invocations successful!")
    print("Response status code:", response.status_code)
    print("Predictions:", response.text)
except requests.RequestException as e:
    print(f"Invocations failed: {e}")
    if 'response' in locals():
        print("Response content:", response.text)

Testing /ping endpoint...
Ping successful! Status code: 200

Testing /invocations endpoint...
Invocations successful!
Response status code: 200
Predictions: [[0.14639568 0.8536043 ]]
[[0.60676062 0.39323938]]



In [55]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.deserializers import JSONDeserializer
import pandas as pd
import json

# Get the SageMaker execution role (assumes this is run in a SageMaker notebook)
role = "arn:aws:iam::794038231401:role/service-role/SageMaker-ExecutionRole-20250103T203496"

# Create a SageMaker session
sagemaker_session = sagemaker.Session()

# Specify your ECR image URI (replace with your actual URI)
image_uri = '794038231401.dkr.ecr.us-east-1.amazonaws.com/my-sagemaker-model:latest'

In [56]:
import tarfile
with tarfile.open('model.tar.gz', 'w:gz') as tar:
    tar.add('docker-folder/model/decision-tree-model.pkl', arcname='decision-tree-model.pkl')

In [57]:
# Upload the dummy file to S3
def_bucket = sagemaker_session.default_bucket()
def_bucket = sagemaker_session.default_bucket()
print(def_bucket)
model_data = sagemaker_session.upload_data(
    path='model.tar.gz', 
    bucket=def_bucket, 
    key_prefix='model')

sagemaker-us-east-1-794038231401


In [58]:
print(model_data)

s3://sagemaker-us-east-1-794038231401/model/model.tar.gz


In [59]:
# Create the SageMaker model
model = Model(
    image_uri=image_uri,
    model_data=model_data,
    role=role,
    sagemaker_session=sagemaker_session
)

# Optional: Delete the endpoint after testing to avoid costs
# predictor.delete_endpoint()

In [60]:
# Deploy the model to an endpoint
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',  # Adjust instance type as needed
    endpoint_name='my-endpoint'   # Replace with a unique endpoint name
)

[03/26/25 21:02:22] INFO     Creating model with name: my-sagemaker-model-2025-03-27-03-02-22-294   ]8;id=226500;file:///home/murivirg/work/github/sagemaker-tutorials/customer_case_using_container/env/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=38915;file:///home/murivirg/work/github/sagemaker-tutorials/customer_case_using_container/env/lib/python3.13/site-packages/sagemaker/session.py#4094\4094]8;;\

[03/26/25 21:02:23] INFO     Creating endpoint-config with name my-endpoint                         ]8;id=711743;file:///home/murivirg/work/github/sagemaker-tutorials/customer_case_using_container/env/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=244185;file:///home/murivirg/work/github/sagemaker-tutorials/customer_case_using_container/env/lib/python3.13/site-packages/sagemaker/session.py#5937\5937]8;;\

                    INFO     Creating endpoint with name my-endpoint                                ]8;id=508541;file:///home/murivirg/work/github/sagemaker-tutorials/customer_case_using_container/env/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=552192;file:///home/murivirg/work/github/sagemaker-tutorials/customer_case_using_container/env/lib/python3.13/site-packages/sagemaker/session.py#4759\4759]8;;\

--------!

In [61]:
print(type(predictor))

<class 'NoneType'>


In [65]:
import pandas as pd
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.deserializers import JSONDeserializer

# Create predictor object for existing endpoint
# Create predictor with CSV deserializer
predictor = Predictor(
    endpoint_name='my-endpoint',
    sagemaker_session=sagemaker.Session(),
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer()  # ← FIX HERE
)

# Prepare test data in the same format as your local test
sample_data = pd.DataFrame({
    'timestamp': ['2023-05-01'],
    'in_data': ['{"yams_score":0.7,"north_star_metric":"5.5"}'],
    'decline_v2a_debit': [0.5],
    'days_since_sms_otp_success': [20],
    'days_since_receiver_first_seen': [100],
    'days_since_device_first_seen': [20],
    'dda_age_in_days': [100]
})

# Convert to CSV WITH HEADERS (index=False, header=True)
csv_data = sample_data.to_csv(index=False, header=True)  # Changed header=False to header=True

# Make prediction
try:
    response = predictor.predict(
        csv_data,
        initial_args={'ContentType': 'text/csv'}
    )
    print("Predictions:", response)
except Exception as e:
    print("Prediction failed:", e)
    if hasattr(e, 'response'):
        print("Error response:", e.response)

Predictions: [['[[0.14639568 0.8536043 ]]'], ['[[0.60676062 0.39323938]]']]
